In [3]:
#LIBRERIAS

# gestion de fechas
from datetime import datetime, timedelta

# gestion de obj
import collections
# api amigocloud
from amigocloud import AmigoCloud

# gestion de reportes
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
from jinja2 import Template
from docx2pdf import convert

# requersts a web
import requests
from werkzeug.wrappers import Request, Response

# general
import os
import time

ModuleNotFoundError: No module named 'amigocloud'

In [3]:
def send_msj(msj):
    now = datetime.now().strftime('%H:%M:%S %d/%m/%Y')
    print(now, '>', msj)

In [4]:
# ruta de carpera donde estan las llaves
send_msj('Estableciendo conexion a AmigoCloud...!!!')
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
# acceso a amigocloud
amigocloud = AmigoCloud(token=api_token)
amigocloud
send_msj('Conectado a AmigoCloud...!!!')

15:27:29 09/10/2023 > Estableciendo conexion a AmigoCloud...!!!
15:27:31 09/10/2023 > Conectado a AmigoCloud...!!!


In [5]:
def generar_reporte(i):
    id_reg = i
    # selecciona registro segun id de registro
    query = {'query': 'select * from dataset_318090 where id = {id_ca}'.format(id_ca=id_reg)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    # cambia formato de fecha dd/mm/aaaa
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    # cambia las observaciones si estan en NONE
    if data['observaciones'] == None:
        data['observaciones']='Sin observaciones'
    # crea el objeto de la inspeccion
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    send_msj('Generando reporte de cañero:' + insp.canhero)
    # seleccionar fotos
    query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_49781 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    # seleccion de template
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_operador-cosechadora.docx")
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['s3_filename'], width=Mm(60))})
    
    # diccionario del conjunto de datos a pasar para reporte
    context = {'insp':insp, 'fotos':lista_fotos_inline}
    # generar el reporte
    doc.render(context)

    # formato de nombre de archivo: "123_EVC_01-01-2022_NOMBRE"
    cod_nom = insp.canhero.split(' / ')
    file_name = ruta + '/informes/_' + str(cod_nom[0]) + '_ECO_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(file_name + '.docx')

In [6]:
lista_id = list(range(320,321))
lista_id

[320]

In [7]:
for i in lista_id:
    print('GENERANDO id = ' + str(i))
    generar_reporte(i)
print('END=====================================')

GENERANDO id = 320
15:27:46 09/10/2023 > Generando reporte de cañero:556 / AYALA ESPINOSA WALDO SELUB
END=====================================


## CONVERTIR A PDF

In [8]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 4 INFORMES
4 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.docx']

In [9]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 4 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END
